# Imports

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import plotly.graph_objs as go
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from utils import TrainTestSequenceDataset, smape, process_for_train, features, targets, spherical_from_cartesian
from models import LSTM, ResNet18
from tqdm.notebook import tqdm
from IPython.display import clear_output, display
import ipywidgets as widgets
import os
from spaceopt import SpaceOpt

# Loading data

In [2]:
data = pd.read_csv('train.csv', parse_dates=['epoch'])
test_sat_id = torch.load('test_sat_id')

# Data processing

In [3]:
sat_datas_train, sat_datas_test = process_for_train(data)

In [4]:
sat_datas_train[0].head()

,epoch,x,y,z,Vx,Vy,Vz,x_sim,y_sim,z_sim,...,fi_sim,dro/dt_sim,dtheta/dt_sim,dfi/dt_sim,dx_sim,dy_sim,dz_sim,dro_sim,dtheta_sim,dfi_sim
0,-1.730756,-8855.823863,13117.780146,-20728.353233,-0.908303,-3.808436,-2.022083,-0.311082,0.500388,-0.999704,...,-1.291078,-0.004444,-0.937090,0.747771,-0.343891,-1.463405,-0.957448,-0.004444,-0.937090,0.747771
1,-1.727132,-10567.672384,1619.746066,-24451.813271,-0.302590,-4.272617,-0.612796,-0.378567,0.051732,-1.180247,...,-1.908559,0.434614,-1.353263,0.125439,-0.114952,-1.642185,-0.291497,0.434614,-1.353263,0.125439
2,-1.723508,-10578.684043,-10180.467460,-24238.280949,0.277435,-4.047522,0.723155,-0.379212,-0.408885,-1.170478,...,1.416777,0.757259,-1.006825,-0.538343,0.104417,-1.556416,0.340189,0.757259,-1.006825,-0.538343
3,-1.719884,-9148.251857,-20651.437460,-20720.381279,0.715600,-3.373762,1.722115,-0.323161,-0.817838,-1.000836,...,0.937656,0.927449,-0.680071,-0.604404,0.270316,-1.298146,0.813059,0.927449,-0.680071,-0.604404
4,-1.716260,-6719.092336,-28929.061629,-14938.907967,0.992507,-2.519732,2.344703,-0.227778,-1.141341,-0.721533,...,0.552240,0.972421,-0.534778,-0.546743,0.375320,-0.970269,1.108234,0.972421,-0.534778,-0.546743


# Model

In [5]:
seq_len = 40
hidden_dim = 300
num_layers = 2
model = LSTM(hidden_dim=hidden_dim, seq_len=seq_len, num_layers=num_layers)
model.train()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001, eps=1e-2)
criterion = smape
n = 1

# Train on n satellite

In [6]:
model.train()
loss_widget = widgets.FloatProgress(min=0, max=1, step=0.01, description='Loss', value=0)  # jupyter widget
display(loss_widget)
train_data = sat_datas_train[n]
x_train = train_data[features]
y_train = train_data[targets]
train_dataset = TrainTestSequenceDataset(x_train, y_train, seq_len=model.seq_len)
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
for epoch in tqdm(range(10)):
    for seq_train_x, seq_train_y in train_dataloader:
        model.zero_grad()  # refresh gradients
        model.init_hidden_cell()
        predictions = model(seq_train_x)
        loss = criterion(predictions, seq_train_y[:, -1, :])
        loss_widget.value = loss
        loss.backward()  # compute gradients
        optimizer.step()  # update network parameters

FloatProgress(value=0.0, description='Loss', max=1.0)

In [7]:
print(predictions)
print(seq_train_y[:, -1, :])

tensor([[-2.9657e+04,  5.3568e+03, -7.0877e+03, -1.0155e+00, -2.0413e+00,
          1.2403e+00]], grad_fn=<CopySlices>)
tensor([[-3.0420e+04,  7.1447e+03, -7.6411e+03, -1.0368e+00, -1.9035e+00,
          1.0968e+00]])


In [8]:
model.eval()
score_widget = widgets.FloatProgress(min=0, max=1, step=0.01, description='Score', value=0)  # jupyter widget
display(score_widget)
test_data = sat_datas_test[n]
x_test = test_data[features]
y_test = test_data[targets]
test_dataset = TrainTestSequenceDataset(x_test, y_test, seq_len=model.seq_len)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)
loss_sum = 0
i = 0

for seq_test_x, seq_test_y in tqdm(test_dataloader):
    with torch.no_grad():
        model.init_hidden_cell()
        predictions = model(seq_test_x)
        loss = criterion(predictions, seq_test_y[:, -1, :])
        loss_sum += loss
        i += 1
        score_widget.value = 1 - loss_sum / i
score =  (1 - loss_sum / i).item()
print(score)

FloatProgress(value=0.0, description='Score', max=1.0)


0.8792967796325684


In [ ]:
def train_model(sat_id, model, optimizer, epoch, criterion=smape):
    model.train()
    loss_widget = widgets.FloatProgress(min=0, max=1, step=0.01, description='Loss', value=0)  # jupyter widget
    display(loss_widget)
    train_data = sat_datas_train[sat_id]
    x_train = train_data[features]
    y_train = train_data[targets]
    train_dataset = TrainTestSequenceDataset(x_train, y_train, seq_len=model.seq_len)
    train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
    mean_loss = 0
    for epoch in tqdm(range(epoch)):
        i = 0
        sum_loss = 0
        if epoch > 2 and mean_loss > 0.5:
            return None
        for seq_train_x, seq_train_y in train_dataloader:
            model.zero_grad()  # refresh gradients
            model.init_hidden_cell()
            predictions = model(seq_train_x)
            loss = criterion(predictions, seq_train_y[:, -1, :])
            loss_widget.value = loss
            loss.backward()  # compute gradients
            optimizer.step()  # update network parameters
            
            i += 1 
            sum_loss += loss
            mean_loss = sum_loss / i

In [ ]:
train_model(sat_id=0, model=model, optimizer=optimizer, criterion=smape, epoch=30)

# Test on n satellite

In [ ]:
def test_model(sat_id, model, optimizer, criterion=smape):
    model.eval()
    score_widget = widgets.FloatProgress(min=0, max=1, step=0.01, description='Score', value=0)  # jupyter widget
    display(score_widget)
    test_data = sat_datas_test[sat_id]
    x_test = test_data[features]
    y_test = test_data[targets]
    test_dataset = TrainTestSequenceDataset(x_test, y_test, seq_len=model.seq_len)
    test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    loss_sum = 0
    i = 0
    
    for seq_test_x, seq_test_y in tqdm(test_dataloader):
        with torch.no_grad():
            model.init_hidden_cell()
            predictions = model(seq_test_x)
            loss = criterion(predictions, seq_test_y[:, -1, :])
            loss_sum += loss
            i += 1
            score_widget.value = 1 - loss_sum / i
    return (1 - loss_sum / i).item()

In [ ]:
test_model(sat_id=0, optimizer=optimizer, model=model)

In [ ]:
def evaluate(model, optimizer, criterion, spoint, sat_id):
    seq_len = spoint['seq_len']
    desc = 'satellite number'
    epoch = spoint['epoch']
    hidden_dim = spoint['hidden_dim']
    num_layers = spoint['num_layers']
    if seq_len > len(sat_datas_test[sat_id]):
        seq_len = sat_datas_test[sat_id]
    
    model = LSTM(hidden_dim=hidden_dim, seq_len=seq_len, num_layers=num_layers)
    optimizer = torch.optim.Adam(params=model.parameters(), lr=spoint['lr'], eps=spoint['eps'])
    criterion = smape
    train_model(sat_id=sat_id, model=model, optimizer=optimizer, criterion=smape, epoch=epoch)
    
    score = test_model(sat_id=sat_id, model=model, optimizer=optimizer, criterion=smape)
    
    return score, model

In [11]:
best_params = {'lr': 0.001, 'eps': 0.0001, 'seq_len': 40, 'hidden_dim': 300, 'epoch': 10, 'num_layers': 2}

In [ ]:
search_space = {
    'lr': [0.001, 0.01, 0.1, 1.],
    'eps': [1e-8, 1e-4, 1e-2, 1e0],
    'seq_len': [10, 20, 30, 40],
    'hidden_dim': [20, 30, 50],
    'epoch': [10],
    'num_layers': [1, 2, 3, 4]
}

In [ ]:
results = {}
spoint = best_params
for sat_id in test_sat_id:
    spaceopt = SpaceOpt(search_space=search_space,
                    target_name='score',
                    objective='max')
    max_score = 0
    score = 0
    iteration = 0
    best_hyperparameters = None
    while score < 0.96 and iteration < 15:
        for iteration in range(4):
            if score > 0.96:
                break
            clear_output()
            print('Previous iteration:')
            print(f'Satellite id: {sat_id}')
            print(f'Iteration: {iteration}')
            print(spoint)
            
            if iteration == 0:
                spoint = spaceopt.get_random()
            elif iteration == 1:
                spoint = best_params
            elif iteration < 3:
                spoint = spaceopt.get_random()   # exploration
            else:
                spoint = spaceopt.fit_predict()  # exploitation
            if sat_id == 252:
                spoint['seq_len'] = 24
            print('This iteration:')
            print(spoint)
            print(model.seq_len)
            score, model= evaluate(spoint=spoint, sat_id=sat_id)
            spoint['score'] = score
            spaceopt.append_evaluated_spoint(spoint)
            
            if score > max_score:
                max_score = score
                best_model = model
                best_hyperparameters = spoint
    else:
        if not os.path.exists(f'models//{sat_id}'):
            os.makedirs(f'models//{sat_id}')
        results[sat_id] = spoint
        torch.save(f'models//{sat_id}//hyperparameters', best_hyperparameters)
        torch.save(f'models//{sat_id}//model.pt', model)

In [12]:
def evaluate_new(spoint, sat_id):
    n = sat_id
    seq_len = spoint['seq_len']
    hidden_dim = spoint['hidden_dim']
    num_layers = spoint['num_layers']
    ep = spoint['epoch']
    lr = spoint['lr']
    eps = spoint['eps']
    
    train_data = sat_datas_train[n]
    if seq_len > int(len(train_data) / 2 - 1):
        seq_len = int(len(train_data) / 2 - 1)
    x_train = train_data[features]
    y_train = train_data[targets]
    
    model = LSTM(hidden_dim=hidden_dim, seq_len=seq_len, num_layers=num_layers)
    model.train()
    optimizer = torch.optim.Adam(params=model.parameters(), lr=lr, eps=eps)
    criterion = smape
    model.train()
    loss_widget = widgets.FloatProgress(min=0, max=1, step=0.01, description='Loss', value=0)  # jupyter widget
    display(loss_widget)
    
    train_dataset = TrainTestSequenceDataset(x_train, y_train, seq_len=model.seq_len)
    train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
    for epoch in tqdm(range(ep)):
        for seq_train_x, seq_train_y in train_dataloader:
            model.zero_grad()  # refresh gradients
            model.init_hidden_cell()
            predictions = model(seq_train_x)
            loss = criterion(predictions, seq_train_y[:, -1, :])
            loss_widget.value = loss
            loss.backward()  # compute gradients
            optimizer.step()  # update network parameters

    model.eval()
    score_widget = widgets.FloatProgress(min=0, max=1, step=0.01, description='Score', value=0)  # jupyter widget
    display(score_widget)
    test_data = sat_datas_test[n]
    x_test = test_data[features]
    y_test = test_data[targets]
    test_dataset = TrainTestSequenceDataset(x_test, y_test, seq_len=model.seq_len)
    test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    loss_sum = 0
    i = 0

    for seq_test_x, seq_test_y in tqdm(test_dataloader):
        with torch.no_grad():
            model.init_hidden_cell()
            predictions = model(seq_test_x)
            loss = criterion(predictions, seq_test_y[:, -1, :])
            loss_sum += loss
            i += 1
            score_widget.value = 1 - loss_sum / i
    score =  (1 - loss_sum / i).item()
    return score, model

In [13]:
results = {}
for sat_id in test_sat_id:
    clear_output()
    print(f'Satellite id: {sat_id}')
    print(results)
    spoint = best_params
    
    score, model= evaluate_new(spoint=spoint, sat_id=sat_id)
    spoint['score'] = score
    if not os.path.exists(f'models//{sat_id}'):
        os.makedirs(f'models//{sat_id}')
    results[str(sat_id)] = spoint
    torch.save(spoint, f'models//{sat_id}//hyperparameters')
    torch.save(model, f'models//{sat_id}//model.pt')

Satellite id: 26
{'1': {'lr': 0.001, 'eps': 0.0001, 'seq_len': 40, 'hidden_dim': 300, 'epoch': 10, 'num_layers': 2, 'score': 0.7898368239402771}, '2': {'lr': 0.001, 'eps': 0.0001, 'seq_len': 40, 'hidden_dim': 300, 'epoch': 10, 'num_layers': 2, 'score': 0.7898368239402771}, '3': {'lr': 0.001, 'eps': 0.0001, 'seq_len': 40, 'hidden_dim': 300, 'epoch': 10, 'num_layers': 2, 'score': 0.7898368239402771}, '4': {'lr': 0.001, 'eps': 0.0001, 'seq_len': 40, 'hidden_dim': 300, 'epoch': 10, 'num_layers': 2, 'score': 0.7898368239402771}, '6': {'lr': 0.001, 'eps': 0.0001, 'seq_len': 40, 'hidden_dim': 300, 'epoch': 10, 'num_layers': 2, 'score': 0.7898368239402771}, '9': {'lr': 0.001, 'eps': 0.0001, 'seq_len': 40, 'hidden_dim': 300, 'epoch': 10, 'num_layers': 2, 'score': 0.7898368239402771}, '16': {'lr': 0.001, 'eps': 0.0001, 'seq_len': 40, 'hidden_dim': 300, 'epoch': 10, 'num_layers': 2, 'score': 0.7898368239402771}, '20': {'lr': 0.001, 'eps': 0.0001, 'seq_len': 40, 'hidden_dim': 300, 'epoch': 10, 'nu

FloatProgress(value=0.0, description='Loss', max=1.0)

FloatProgress(value=0.0, description='Score', max=1.0)

ValueError: __len__() should return >= 0

In [ ]:
score, model = evaluate(sat_id=0, spoint=best_params)

In [ ]:
sched = AsyncHyperBandScheduler(
        time_attr="training_iteration", metric="mean_accuracy")
    analysis = tune.run(
        train_mnist,
        name="exp",
        scheduler=sched,
        stop={
            "mean_accuracy": 0.98,
            "training_iteration": 5 if args.smoke_test else 100
        },
        resources_per_trial={
            "cpu": 2,
            "gpu": int(args.cuda)
        },
        num_samples=1 if args.smoke_test else 50,
        config={
            "lr": tune.sample_from(lambda spec: 10**(-10 * np.random.rand())),
            "momentum": tune.uniform(0.1, 0.9),
            "use_gpu": int(args.cuda)
        })

    print("Best config is:", analysis.get_best_config(metric="mean_accuracy"))

# Good parameters

In [ ]:
point1 = {'lr': 0.1, 'eps': 0.01, 'seq_len': 5, 'hidden_dim': 30, 'epoch': 60}
point2 = {'lr': 1.0, 'eps': 1.0, 'seq_len': 20, 'hidden_dim': 30, 'epoch': 60}
point3 = {'lr': 0.01, 'eps': 0.0001, 'seq_len': 30, 'hidden_dim': 50, 'epoch': 60}  # super good

In [ ]:
score = 0
best_spoint = None
for spoint in spaceopt.evaluated_spoints:
    if spoint['score'] > score:
        score = spoint['score']
        best_spoint = spoint

In [ ]:
best_spoint

In [ ]:
model.eval()
y_pred = model(x_test)
after_train = criterion(y_pred.squeeze(), y_test) 
print('Test loss after Training' , after_train.item())

In [ ]:
y_pred.shape